# Download Landsat 7-9 image subsets over an AOI using AWS

_Last modified 2023-11-1._

This script is run to download Landsat images over the glaciers available through the Amazon Web Services (AWS) s3 bucket. The workflow is streamlined to analyze images for 10s to 100s of glaciers, specifically, the marine-terminating glaciers along the periphery of Greenland. Sections of code that may need to be modified are indicated as below:

    ##########################################################################################

    code to modify

    ##########################################################################################

 
### First, configure your AWS profile to access the Landsat images on the s3 bucket:

Follow instructions at https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-install.html to get required __aws command line software__.

Set up your AWS profile with a payment account. Then configure it to your machine following these steps:

    aws configure --profile terminusmapping
    
Enter in your credentials, which will be stored locally on your computer. The Boto3 package will be used to access your credentials without leaking your keys. **Protect your AWS access keys!** DO NOT print anything that involves your ACCESS_KEY and SECRET_KEY. GitGuardian may help track any leaked keys. In order to use these credentials to download subsets of images from AWS using vsi3, **GDAL version 3.2. or newer must be installed**.

In [1]:
# AWS settings
import boto3
import boto3.session

cred = boto3.Session(profile_name='terminusmapping').get_credentials()
ACCESS_KEY = cred.access_key
SECRET_KEY = cred.secret_key
SESSION_TOKEN = cred.token  ## optional


s3client = boto3.client('s3', 
                        aws_access_key_id = ACCESS_KEY, 
                        aws_secret_access_key = SECRET_KEY, 
#                         aws_session_token = SESSION_TOKEN
                       )

######################################################################################
# path to the collection on AWS usgs-landsat s3 bucket:
collectionpath = 'collection02/level-1/standard/' # collection 2 level 1 data being used (contains B8)
######################################################################################

# 1) Set-up: import packages and set paths


In [19]:
import numpy as np
import pandas as pd
import math
import subprocess
import os
import shutil
import datetime
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob

# geospatial packages
import fiona
import geopandas as gpd
from shapely.geometry import Polygon, Point, LineString
import shapely
from matplotlib.pyplot import imshow

# Enable fiona KML file reading driver
fiona.drvsupport.supported_drivers['KML'] = 'rw'

# distance function
def distance(x1, y1, x2, y2):
    dist = (((x2-x1)**2)+((y2-y1)**2))**(1/2)
    return dist

### Define paths, satellites, geographic projections:

In [4]:
######################################################################################
# ADJUST THESE VARIABLES:
# basepath = '/Users/jukesliu/Documents/TURNER/DATA/shapefiles_gis/' # folder containing the glacier shapefile(s)
# downloadpath ='/Users/jukesliu/Documents/TURNER/DATA/IMAGERY/LSimages/' # folder to eventually contain downloaded Landsat images
basepath = '/Users/jukesliu/Documents/PLANETSCOPE_VELOCITIES/' # folder containing the glacier shapefile(s)
downloadpath ='/Volumes/JUKES_EXT/CautoRIFT_sites/LS_LO/' # folder to eventually contain downloaded Landsat images

sats = ['L8','L9'] # names of landsats to download images from ('L7' for Landsat 7 or 'L8' or both)
# sats = ['L7']
L9_yrs = np.arange(2018,2025).astype(str) # set target years for L9: 2020-2021
L8_yrs = np.arange(2018,2025).astype(str) # set target years for L8: 2013-2021
L7_yrs = np.arange(1999,2003).astype(str) # set target years for L7: 1999-2003
L9_bands = [8] # panchromatic band for L9
L8_bands = [8] # panchromatic band for L8
L7_bands = [8] # panchromatic band for L7
######################################################################################

In [5]:
## Read in AOI shapefile and determine source_srs ####################################
AOIpath = basepath+'LO_Box_WGS.shp'
# this path is also where the UTM reprojections of the AOI shp will be stored
AOI = fiona.open(AOIpath).next()
aoi = AOI['geometry']['coordinates'][0]
print(aoi)

source_srs = '4326' # EPSG code for the current projection of the glacier shapefile(s)

# Reproject to WGS if in a different srs
# boxespath = basepath+"Box"+BoxID+"/Box"+BoxID # access the BoxID folders created 
# # construct the gdal command
# rp = "ogr2ogr -f 'ESRI Shapefile' -t_srs EPSG:4326 -s_srs EPSG:"+source_srs+" "
# rp +=boxespath+"_WGS.shp "+boxespath+".shp"
# print("Command:", rp) # check command
# subprocess.run(rp, shell=True, check=True) # run the command on terminal

# # if an error is produced, check the error output on the terminal window that runs this notebook
######################################################################################

[(-138.64580366321644, 60.453832384824246), (-137.84840393795992, 60.45537151123912), (-137.86668806486244, 60.120521992664955), (-138.65597158985227, 60.1188431719691), (-138.64580366321644, 60.453832384824246)]


/var/folders/mr/7f69f7y50zj85n420pyvh8cr0000gq/T/ipykernel_59884/1259367196.py:4: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  AOI = fiona.open(AOIpath).next()



# 2) Find all the Landsat footprints that overlap the glaciers

This step requires the **WRS-2_bound_world_0.kml** file containing the footprints of all the Landsat scene boundaries available through the USGS (https://www.usgs.gov/media/files/landsat-wrs-2-scene-boundaries-kml-file). Place this file in your base directory (basepath). 

To check if they overlap the glacier terminus box shapefiles, the box shapefiles must be in WGS84 coordinates (ESPG: 4326). If they are not yet, we use the following GDAL command to reproject them into WGS84:

        ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:NEW_EPSG_NUMBER -s_srs EPSG:OLD_EPSG_NUMBER out.shp in.shp

In [8]:
######################################################################################
# open the kml file with the Landsat path, row footprints:
WRS = fiona.open('/Volumes/JUKES_EXT/WRS-2_bound_world_0.kml', driver='KML') # check the path to the world bounds file
print('Landsat footprint file opened.')
######################################################################################

Landsat footprint file opened.


In [9]:
# Grab the WGS84 coordinates of the AOI
points = [] # to hold the box vertices
# read coordinates and convert to a shapely object
for coord_pair in list(aoi): 
    lat = coord_pair[0]; lon = coord_pair[1]        
    point = shapely.geometry.Point(lat, lon) # create shapely point 
    points.append(point) # append to points list
print("coordinates recorded.") # keep track of progress

coordinates recorded.


In [10]:
paths = []; rows = []; boxes = [] # create lists to hold the paths and rows and BoxIDs

#loop through all Landsat scenes (path, row footprints)
for feature in WRS:
    # create shapely polygons from the Landsat footprints
    coordinates = feature['geometry']['coordinates'][0]
    coords = [xy[0:2] for xy in coordinates]
    pathrow_poly = Polygon(coords)
    
    # grab the path and row name from the WRS kml file:
    pathrowname = feature['properties']['Name']  
    path = pathrowname.split('_')[0]; row = pathrowname.split('_')[1]
#     print(path, row)
      
    box_points_in = 0 # counter for number of box_points in the pathrow_geom:
    for i in range(0, len(points)):
        point = points[i]
        if point.within(pathrow_poly): # if the pathrow shape contains the point
            box_points_in = box_points_in+1 # append the counter
    if box_points_in == 5: # if all box vertices are inside the footprint, save the path, row, BoxID
        paths.append('%03d' % int(path))
        rows.append('%03d' % int(row))

# Store in dataframe
boxes_pr_df = pd.DataFrame(list(zip(paths, rows)), columns=['Path', 'Row'])
boxes_pr_df # display

,Path,Row
0,061,018
1,060,018


# 3) Download metadata files from AWS s3 for overlapping Landsat scenes
     
The syntax for listing the Collection 2 Landsat image files AWS s3 bucket is as follows:

    aws s3 ls --profile terminusmapping --request-payer requester s3://usgs-landsat/collection02/level-2/standard/oli-tirs/yyyy/path/row/LC08_LS2R_pathrow_yyyyMMdd_yyyyMMdd_02_T1/ 
    
__NOTE: Including the --request-payer requester as part of this line indicates that the referenced user will be charged for data download.__

We can use the paths and rows in the dataframe to access the full Landsat scene list and the corresponding metdata files. Read https://docs.opendata.aws/landsat-pds/readme.html to learn more.
    
The metadata files will be downloaded into folders corresponding to the Landsat footprint, identified by the Path Row numbers:
    
    aws s3api get-object --bucket usgs-landsat --key collection02/level-2/standard/oli-tirs/yyyy/path/row/LC08_L2SP_pathrow_yyyyMMdd_yyyyMMdd_02_T1/LC08_L2SP_pathrow_yyyyMMdd_yyyyMMdd_02_T1_MTL.txt  --request-payer requester LC08_L2SP_pathrow_yyyyMMdd_yyyyMMdd_02_T1_MTL.txt

In [11]:
# Loop through the dataframe containing overlapping path, row info:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; r = row['Row']; folder_name = 'Path'+p+'_Row'+r+'_c2' # folder name
    bp_out = downloadpath+folder_name+'/' # output path for the downloaded files
    print("Downloaded metadata files are stored in:",bp_out)
    
    # create Path_Row folders if they don't exist already
    if os.path.exists(bp_out):
        print(folder_name, " exists already, skip directory creation")
    else:
        os.mkdir(bp_out)
        print(folder_name+" directory made")
    
    for sat in sats: # for each satellite
        if sat == 'L8':
            collectionfolder = 'oli-tirs/'; years = L8_yrs; prefix='LC08' # set folder, years, file prefix
        elif sat == 'L7':
            collectionfolder = 'etm/'; years = L7_yrs; prefix='LE07' # set folder, years, file prefix
        elif sat == 'L9':
            collectionfolder = 'oli-tirs/'; years = L8_yrs; prefix='LC09' # set folder, years, file prefix
            
        # loop through years
        for year in years:
            # grab list of images in each year, path, row folder
            find_imgs = 'aws s3 ls --profile terminusmapping --request-payer requester s3://usgs-landsat/'
            find_imgs += collectionpath+collectionfolder
            find_imgs += year+'/'+p+'/'+r+'/'

            if subprocess.run(find_imgs,shell=True).returncode != 0:
                print('No results found for '+collectionpath+collectionfolder+year+'/'+p+'/'+r+'/')
                results = [] # empty results
            else:
                result = subprocess.check_output(find_imgs,shell=True) # grab the avilable images
                results = result.split() # split string
            
            imagenames = []
            for line in results: # loop through strings
                line = str(line)
                if prefix in line and 'T1' in line: # find just the Tier-1 images
                    imgname = line[2:-2]; imagenames.append(imgname)

            # download the metadata (MTL.txt) file if it doesn't exist
            for imgname in imagenames:
                if not os.path.exists(bp_out+imgname+'_MTL.txt'): # check in output directory
                    command = 'aws s3api get-object --bucket usgs-landsat --key '+collectionpath+collectionfolder
                    command += year+'/'+p+'/'+r+'/'
                    command += imgname+'/'+imgname+'_MTL.txt'
                    command += ' --profile terminusmapping --request-payer requester '
                    command += bp_out+imgname+'_MTL.txt'
                    print('Downloading', imgname+'_MTL.txt')
                    subprocess.run(command,shell=True,check=True)
                else:
                    print(imgname+'_MTL.txt exists. Skip.')

Downloaded metadata files are stored in: /Volumes/JUKES_EXT/CautoRIFT_sites/LS_LO/Path061_Row018_c2/
Path061_Row018_c2 directory made
                           PRE LC08_L1GT_061018_20180115_20200902_02_T2/
                           PRE LC08_L1GT_061018_20180320_20200901_02_T2/
                           PRE LC08_L1GT_061018_20181014_20200830_02_T2/
                           PRE LC08_L1TP_061018_20180131_20200902_02_T1/
                           PRE LC08_L1TP_061018_20180216_20200902_02_T1/
                           PRE LC08_L1TP_061018_20180304_20200902_02_T1/
                           PRE LC08_L1TP_061018_20180405_20200901_02_T1/
                           PRE LC08_L1TP_061018_20180421_20200901_02_T1/
                           PRE LC08_L1TP_061018_20180507_20200901_02_T1/
                           PRE LC08_L1TP_061018_20180523_20200901_02_T1/
                           PRE LC08_L1TP_061018_20180608_20200831_02_T1/
                           PRE LC08_L1TP_061018_20180624_202008

# 4) Download QAPIXEL band over terminus box to determine cloud cover

If the terminus box shapefiles were not originally in UTM projection, will need to reproject them into UTM to match the Landsat projection. The code automatically finds the UTM zones from the metadata files and fills in the following syntax to reproject:
    
    ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:326zone output.shp input.shp
    
#### If the terminus box shapefiles are already in UTM projection, skip the following cell and rename the files to end with "\_UTM\_##.shp" where ## corresponds to the zone number (e.g., "\_UTM\_07.shp", "\_UTM\_21.shp").

In [12]:
# If pyproj is not showing up:
os.environ['PROJ_LIB'] = '/Users/jukesliu/opt/anaconda3/envs/autoterm_env/share/proj'

In [13]:
zones = {} # initialize dictionary to hold UTM zone for each Landsat scene path row
zone_list = [] # list of zones

# Loop through all scenes:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; r = row['Row']; folder_name = 'Path'+p+'_Row'+r+'_c2' # Landsat path and row
    pr_folderpath = downloadpath+folder_name+'/' # path to the downloaded metadata files
    
    if len(os.listdir(pr_folderpath)) > 0: # if there are files in the folder
        # grab UTM Zone from the first metadata file
        mtl_scene = glob.glob(pr_folderpath+'*_MTL.txt')[0]
        mtl = open(mtl_scene, 'r')
        
        # loop through lines in the metadata file to find the UTM ZONE
        for line in mtl:  
            variable = line.split("=")[0] # grab the variable name
            if ("UTM_ZONE" in variable):
                zone = '%02d' % int(line.split("=")[1][1:-1]) # grab the 2-digit zone number
                zones.update({folder_name: zone}); zone_list.append(zone) # add to zone lists
                break
                
        # reproject shapefile(s) into UTM
        zone = zones[folder_name]
        rp_shp = 'ogr2ogr -f "ESRI Shapefile" '+AOIpath[:-4]+'_UTM_'+zone+'.shp '+AOIpath
        rp_shp += ' -t_srs EPSG:326'+zone
        subprocess.run(rp_shp, shell=True,check=True)
        
    else: # if no files in folder, zone = nan, must fill in manually
        zone_list.append(np.nan)
        
boxes_pr_df['Zone'] = zone_list # add to the path row dataframe
boxes_pr_df.head()

,Path,Row,Zone
0,061,018,07
1,060,018,08


Use GDAL and __vsi3__ link to download subset of the quality band we will use to determine cloud cover over the terminus:

    gdalwarp -cutline path_to_shp.shp -crop_to_cutline /vsi3/usgs-landsat/collection02/level-1/standard/oli-tirs/yyyy/path/row/scene/scene_QA_PIXEL.TIF path_to_subset_QA_PIXEL.TIF


In [14]:
# Loop through all scenes:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; r = row['Row']; zone = row['Zone'] # grab path, row, zone
    folder_name = 'Path'+p+'_Row'+r+'_c2'
    pr_folderpath = downloadpath+folder_name+'/' # path to the downloaded metadata files
    pathtoshp_rp = AOIpath[:-4]+'_UTM_'+zone # path to the UTM projected box shapefile

    files = os.listdir(pr_folderpath) # grab the names of the Landsat scenes
    
    # for all files in the path row folders
    for file in files:
        scene = file[:40] # slice the filename to grab the scene name

        if scene.startswith('L') and 'T1' in scene: # L1TP scenes
            scene_year = scene[17:21] # grab the year from the scene name
            
            if scene.startswith('LC08') or scene.startswith('LC09'):
                collectionfolder='oli-tirs/'
            elif scene.startswith('LE07'):
                collectionfolder='etm/'
                
            # set path to the QA pixel Landsat files
            pathtoQAPIXEL='/vsis3/usgs-landsat/'+collectionpath+collectionfolder
            pathtoQAPIXEL+=scene_year+'/'
            pathtoQAPIXEL+=p+'/'+r+'/'
            pathtoQAPIXEL+=scene+'/'+scene+"_QA_PIXEL.TIF"
            
            # set path to the subset QA pixel files inside the path row folders
            subsetout = pr_folderpath+scene+'_QA_PIXEL.TIF' 
            
            # if the file hasn't already been downloaded
            if not os.path.exists(subsetout): #and not scene_year.startswith('202'): # EXCLUDE CERTAIN YEARS
                print('Downloading', scene)
                # construct download command
                QAPIXEL_dwnld_cmd='gdalwarp -overwrite -cutline '+pathtoshp_rp+'.shp -crop_to_cutline '
                QAPIXEL_dwnld_cmd+= pathtoQAPIXEL+' '+subsetout
                QAPIXEL_dwnld_cmd+=' --config AWS_REQUEST_PAYER requester --config AWS_REGION us-west-2'
                QAPIXEL_dwnld_cmd+=' --config AWS_SECRET_ACCESS_KEY '+SECRET_KEY
                QAPIXEL_dwnld_cmd+=' --config AWS_ACCESS_KEY_ID '+ACCESS_KEY

                subprocess.run(QAPIXEL_dwnld_cmd, shell=True, check=True)

Creating output file that is 1486P x 1310L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2023/061/018/LC08_L1TP_061018_20230302_20230316_02_T1/LC08_L1TP_061018_20230302_20230316_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1486P x 1310L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2022/061/018/LC09_L1TP_061018_20220830_20230331_02_T1/LC09_L1TP_061018_20220830_20230331_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1486P x 1310L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2018/061/018/LC08_L1TP_061018_20180421_20200901_02_T1/LC08_L1TP_061018_20180421_20200901_02_T1_QA_PIXEL.TIF [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 1486P x 1310L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2023/061/018/

# 5) Download non-cloudy Landsat images from AWS

To remove cloudy images, we will find the number of pixels in our terminus box that exceed a threshold value in the QA_PIXEL band corresponding to cloud and cloud shadow likelihood. If the fraction of cloudy pixels with values is above the threshold, we won't download the image. See Landsat Collection 2 Level 2 Science Product Guide for [Landsat 8](https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/media/files/LSDS-1619_Landsat-8-9-C2-L2-ScienceProductGuide-v4.pdf) and [Landsat 7](https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/LSDS-1618_Landsat-4-7_C2-L2-ScienceProductGuide-v3.pdf) more information on how the QA_PIXEL threshold values are chosen.

Additionally, we remove images that are primarily black (fill value of 1 in QA_PIXEL band). This ensures that the scenes that cut off halfway across the glacier are not included in further analysis. The fill percent threshold may need to be adjusted.

In [15]:
######################################################################################
# These are the recommended values based on the Collection 2 Level 2 Science Product Guide.
# Adjust thresholds here:
QAPIXEL_thresh_lower_L7 = 5696.0 # minimum QA pixel value threshold to be considered cloud for L7 images
QAPIXEL_thresh_upper_L7 = 7568.0 # maximum QA pixel value threshold to be considered cloud for L7 images

# Landsat 8 requires two lower thresholds, we selct between 22280 and 24472 and above 54596
QAPIXEL_thresh_lower_L8 = 22080.0 # minimum QA pixel value threshold to be considered cloud for L8 images
QAPIXEL_thresh_upper_L8 = 30048.0 # maximum QA pixel value threshold to be considered cloud for L8 images
QAPIXEL_thresh2_lower_L8 = 54596.0 # 2nd minimum QA pixel value threshold to be considered cloud for L8 images

cpercent_thresh = 100.0 # maximum cloud cover % in terminus box
fpercent_thresh = 60.0 # maximum fill % in terminus box
######################################################################################

In [16]:
# Download images that pass these thresholds:
for index, row in boxes_pr_df.iterrows():
    # grab paths
    p = row['Path']; zone = row['Zone']; r = row['Row']
    folder_name = 'Path'+p+'_Row'+r+'_c2'
    pr_folderpath = downloadpath+folder_name+'/'
    bp_out = downloadpath
    
    print(pr_folderpath)
    
    # path to the shapefile covering the region that will be downloaded
    pathtobuffer = AOIpath[:-4]+'_UTM_'+zone+'.shp' # just the box
    
    for scene in os.listdir(pr_folderpath):
        if scene.startswith('L') and scene.endswith(".TIF") and 'T1' in scene and 'L1TP' in scene: # For Tier-1 images
            scene = scene[:40] # scene name
            year = scene[17:21] # grab acquisition year
            
            QApixelpath = pr_folderpath+scene+'_QA_PIXEL.TIF' # path to QA_PIXEL file
            subsetQApixel = mpimg.imread(QApixelpath) # read in QAPIXEL file as numpy array
            totalpixels = subsetQApixel.shape[0]*subsetQApixel.shape[1] # count total number of pixels
            
            if scene.startswith("LC08") or scene.startswith('LC09'): # Landsat 8 or Landsat 9
                collectionfolder = 'oli-tirs/'; bands = L8_bands; 
                # countcloudy pixels based on thresholds:
                cloudQApixel = subsetQApixel[((subsetQApixel >= QAPIXEL_thresh_lower_L8) &
                                             (subsetQApixel < QAPIXEL_thresh_upper_L8) | 
                                             (subsetQApixel >= QAPIXEL_thresh2_lower_L8))]
                
            elif scene.startswith("LE07"): # Landsat 7
                collectionfolder = 'etm/'; bands = L7_bands
                # countcloudy pixels based on thresholds:
                cloudQApixel = subsetQApixel[((subsetQApixel >= QAPIXEL_thresh_lower_L7) & 
                                             (subsetQApixel < QAPIXEL_thresh_upper_L7))]
 
            # calculate percentages of cloud and fill pixels
            fillQApixel = subsetQApixel[subsetQApixel < 2.0] # fill pixels (value = 0 or 1)
            cloudpixels = len(cloudQApixel); fillpixels = len(fillQApixel) # count the cloudy and fill pixels
            cloudpercent = int(float(cloudpixels)/float(totalpixels)*100) # calculate percent cloudy
            fillpercent = int(float(fillpixels)/float(totalpixels)*100) # calculate percent fill
            
            # evaluate thresholds
            if cloudpercent <= cpercent_thresh and fillpercent <= fpercent_thresh:
                # download the bands for that scene into your scene folders:
                for band in bands:
                        band = str(band) # string format
                        
                        # input path to your bands in AWS:
                        pathin = '/vsis3/usgs-landsat/'+collectionpath+collectionfolder+year+'/'+p+"/"+r+"/"+scene+"/"+scene+"_B"+band+".TIF"
                        
                        outfilename = scene+"_B"+band+'.TIF' # output file name
                        pathout = downloadpath+outfilename # full output file path
                        
                        # if the file hasn't already been downloaded
                        if not os.path.exists(pathout):
                            # download
                            download_cmd = 'gdalwarp -overwrite -cutline '+pathtobuffer+' -crop_to_cutline '+pathin+' '+pathout
                            download_cmd+=' --config AWS_REQUEST_PAYER requester --config AWS_REGION us-west-2'
                            download_cmd+=' --config AWS_SECRET_ACCESS_KEY '+SECRET_KEY
                            download_cmd+=' --config AWS_ACCESS_KEY_ID '+ACCESS_KEY   
                            print('Downloading:', outfilename)
                            subprocess.run(download_cmd, shell=True, check=True)
                        else:
                            print(outfilename, 'exists')
            else:
                print(scene, 'failed cloud & fill thresholds: Cloud % ', cloudpercent, 'Fill %', fillpercent)

/Volumes/JUKES_EXT/CautoRIFT_sites/LS_LO/Path061_Row018_c2/
Downloading: LC08_L1TP_061018_20190203_20200829_02_T1_B8.TIF
Creating output file that is 2972P x 2620L.
Processing /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/061/018/LC08_L1TP_061018_20190203_20200829_02_T1/LC08_L1TP_061018_20190203_20200829_02_T1_B8.TIF [1/1] : 0Using internal nodata values (e.g. 0) for image /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/061/018/LC08_L1TP_061018_20190203_20200829_02_T1/LC08_L1TP_061018_20190203_20200829_02_T1_B8.TIF.
Copying nodata values from source /vsis3/usgs-landsat/collection02/level-1/standard/oli-tirs/2019/061/018/LC08_L1TP_061018_20190203_20200829_02_T1/LC08_L1TP_061018_20190203_20200829_02_T1_B8.TIF to destination /Volumes/JUKES_EXT/CautoRIFT_sites/LS_LO/LC08_L1TP_061018_20190203_20200829_02_T1_B8.TIF.
...10...20...30...40...50...60...70...80...90...100 - done.
Downloading: LC08_L1TP_061018_20190526_20200828_02_T1_B8.TIF
Creating output file th

# 6) Delete all quality band files (*QA_PIXEL.TIF) to save space

These files will not be needed after the download step, so they can be removed to save space.

In [18]:
# Grab all path row folder names from boxes_pr_df:
paths = boxes_pr_df['Path']; rows = boxes_pr_df['Row']
for a in range(0, len(paths)): # look in each path row folder
    folder_name = 'Path'+paths[a]+'_Row'+rows[a]+'_c2'
    folderpath = downloadpath+folder_name+'/'

    # remove all files with QA_PIXEL in the name
    for file in os.listdir(folderpath):
        if 'QA_PIXEL' in file:
            os.remove(folderpath+file)